In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import sys
sys.path.append("../src") if "../src" not in sys.path else None # avoid adding multiple relave paths to sys.path

In [3]:
from Agent import Agent
from Environment import Rectangle

In [4]:
boxsize=(2.2, 2.2)
origo=(0,0)
soft_boundary=0.2

In [5]:
env = Rectangle(boxsize=boxsize, soft_boundary=soft_boundary)
avoid_walls = env.avoid_walls
env.add_wall("free_wall", {"bias":np.array([0.4,0.4]),"slope":np.array([0.8,0.8])})

In [ ]:
ag = Agent(np.pi, 0.02, np.array([[0.5,0.51]]))

for i in range(2000):
    ag.step(avoid_walls)

print("------------")
print(ag.speeds)
print(ag.hds)
print(ag.velocities)
print(ag.positions.shape)

In [ ]:
fig,axs=plt.subplots(figsize=(10,10))

for wallname,wall in env.walls.items():
    # plottable wall matrix
    W = np.array([wall['bias'],sum(wall.values())]).T
    
    # wall
    axs.plot(*W,'orange')
    
    if "free_wall" in wallname:
        # The normal vectors n1,n2 to a vector v=(vx,vy)=(v[0],v[1]) in R^2
        # is the vectors n1 = (-vy,vx) = (-v[1],v[0]) and n2 = (vy,-vx) = (v[1],-v[0])
        # For a more in depth explanation, see:
        # https://stackoverflow.com/questions/1243614/how-do-i-calculate-the-normal-vector-of-a-line-segment
        v = W.T[1] - W.T[0]
        n1 = np.array([-v[1],v[0]])
        n1 = n1 / euclidean([0,0],n1) * soft_boundary# unit normal
        n2 = np.array([v[1],-v[0]])
        n2 = n2 / euclidean((0,0),n2) * soft_boundary# unit normal
        
        SB1 = (W.T + n1).T # soft boundary 1
        axs.plot(*SB1,'orange')
        SB2 = (W.T + n2).T # soft boundary 2
        axs.plot(*SB2,'orange')
    else: # boundary wall
        # soft boundary
        axs.plot(*np.abs(W - soft_boundary),'orange')
        
        
"""
# reformat walls as easily plottable matrices
W1 = np.array([env.walls['w1']['bias'],sum(env.walls['w1'].values())]).T
W2 = np.array([env.walls['w2']['bias'],sum(env.walls['w2'].values())]).T
W3 = np.array([env.walls['w3']['bias'],sum(env.walls['w3'].values())]).T
W4 = np.array([env.walls['w4']['bias'],sum(env.walls['w4'].values())]).T

# walls plot
axs.plot(*W1,'orange')
axs.plot(*W2,'orange')
axs.plot(*W3,'orange')
axs.plot(*W4,'orange')


# soft boundary plot
axs.plot(*np.abs(W1 - soft_boundary),'orange')
axs.plot(*np.abs(W2 - soft_boundary),'orange')
axs.plot(*np.abs(W3 - soft_boundary),'orange')
axs.plot(*np.abs(W4 - soft_boundary),'orange')
"""

"""
# hard coded walls
plt.plot((0,0),(0,2.2),'orange')
plt.plot((0,2.2),(0,0),'orange')
plt.plot((2.2,2.2),(0,2.2),'orange')
plt.plot((2.2,0),(2.2,2.2),'orange')
"""

# plot animal path
n=ag.positions.shape[0]
c=np.zeros((n,4))
c[:,-1] = 1
c[:,:-1] = 0.9-np.linspace(0,0.9,n)[:,None]
axs.scatter(*ag.positions.T,s=0.1,c=c)#[(0,0,0,1)]*100001)



# Testing whether numpy.append or list.append is faster

In [93]:
from ctimeit import ctimeit

In [100]:
@ctimeit
def array_cat():
    a = np.zeros(0)
    for i in range(100000):
        a = np.append(a,i)
    
    return a

array_cat()

File "<ipython-input-100-d38cc1da07f1>", line 9, function: array_cat() used <2.150149049999527> seconds


array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 9.9997e+04, 9.9998e+04,
       9.9999e+04])

In [103]:
@ctimeit
def list_cat():
    a = []
    for i in range(100000):
        a.append(i)
    
    return i
array_cat()

File "<ipython-input-103-d0af6f9751c8>", line 8, function: array_cat() used <2.2002655910000612> seconds


array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 9.9997e+04, 9.9998e+04,
       9.9999e+04])

# Conclusion: NO DIFFERENCE